In [1]:
#run if havent done so in terminal
pip install pandas
pip install pprint
pip install numpy
pip install praw

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3284255841.py, line 2)

In [1]:
import pandas as pd
from pprint import pprint
import numpy as np

In [4]:
#authentication and user agent is description of project
#Put your keys here
import praw

user_agent = "Scraper by me" 

reddit = praw.Reddit(
    client_id = "umwjs-jFpZ37B5Sx669QhA",
    client_secret = "_s7BHD2a6_pxEbprOYfWSX-H7Lg3vw",
    user_agent = user_agent
)

In [5]:
#can type .hot .top .new or .rising but using hot because it is recent data & top is top of all time
#can also use subreddit conservaitve or other
#limit can limit number of posts and print shows how many headlines you got 
headlines = set()
for submission in reddit.subreddit('conservative').hot(limit = None):
    headlines.add(submission.title)
print(len(headlines))

698


In [6]:
df = pd.DataFrame(headlines)
df.head()

,0
0,Protesters Block Street Outside Hotel Where Co...
1,"""Unlike Trump I Don't Shock Allies"": Biden On ..."
2,Michigan school shooting: Crumbley lawyer says...
3,Sarah Silverman slams Joy Reid as media panics...
4,New York City Implementing COVID Vaccine Manda...


In [7]:
df.to_csv('headlines.csv', header = False, encoding = 'utf-8', index=False)

In [8]:
# if nltk does not download package use this answer and instal vader_lexicon package
# https://stackoverflow.com/questions/38916452/nltk-download-ssl-certificate-verify-failed

In [9]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [10]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/alex/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [11]:
#runs sentiment analysis and checks if it work with 3 entries
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in headlines:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)
    
pprint(results[:3], width=100)

[{'compound': -0.5859,
  'headline': 'Protesters Block Street Outside Hotel Where Conservative Activist Was Speaking',
  'neg': 0.375,
  'neu': 0.625,
  'pos': 0.0},
 {'compound': -0.3434,
  'headline': '"Unlike Trump I Don\'t Shock Allies": Biden On Expanding Omicron Travel Ban',
  'neg': 0.244,
  'neu': 0.609,
  'pos': 0.148},
 {'compound': -0.3818,
  'headline': 'Michigan school shooting: Crumbley lawyer says gun was locked up, accuses '
              'prosecutor of creating media circus',
  'neg': 0.24,
  'neu': 0.65,
  'pos': 0.11}]


In [12]:
#Create dataframe of results and show beginning of the dataframe
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.375,0.625,0.000,-0.5859,Protesters Block Street Outside Hotel Where Co...
1,0.244,0.609,0.148,-0.3434,"""Unlike Trump I Don't Shock Allies"": Biden On ..."
2,0.240,0.650,0.110,-0.3818,Michigan school shooting: Crumbley lawyer says...
3,0.140,0.676,0.184,0.2263,Sarah Silverman slams Joy Reid as media panics...
4,0.000,1.000,0.000,0.0000,New York City Implementing COVID Vaccine Manda...


In [13]:
#Labeling as positve or negative given values 
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

,neg,neu,pos,compound,headline,label
0,0.375,0.625,0.000,-0.5859,Protesters Block Street Outside Hotel Where Co...,-1
1,0.244,0.609,0.148,-0.3434,"""Unlike Trump I Don't Shock Allies"": Biden On ...",-1
2,0.240,0.650,0.110,-0.3818,Michigan school shooting: Crumbley lawyer says...,-1
3,0.140,0.676,0.184,0.2263,Sarah Silverman slams Joy Reid as media panics...,1
4,0.000,1.000,0.000,0.0000,New York City Implementing COVID Vaccine Manda...,0


In [14]:
#Creating new dataframe to get ride of everything except headline and label 
df2 = df[['headline', 'label']]

In [15]:
df2.to_csv('reddit_headlines_labels.csv', encoding = 'utf-8', index=False)

In [16]:
#Shows count of how many positive and negative headlines
df.label.value_counts()

-1    304
 0    268
 1    126
Name: label, dtype: int64

In [17]:
#Shows the percentage of each type of headline
df.label.value_counts(normalize=True) * 100

-1    43.553009
 0    38.395415
 1    18.051576
Name: label, dtype: float64

In [18]:
print("Positive headlines:\n")
pprint(list(df[df['label'] == 1].headline)[:5], width=200)

print("\nNegative headlines:\n")
pprint(list(df[df['label'] == -1].headline)[:5], width=200)


Positive headlines:

["Sarah Silverman slams Joy Reid as media panics about DeSantis' proposed civilian military force in Florida",
 'Dec 2, 2021 - Canadian video platform Rumble, popular with right-wing users, will go public via SPAC at a $2.1B valuation and expects to raise around $400M (Radhika Anilkumar/Reuters)... '
 'https://www.reuters.com/markets/funds/video-platform-rumble-go-public-via-21-bln-spac-deal-2021-12-02/',
 "German Health Minister: 'We Must Take Large Groups Of Unvaccinated And Concentrate Them Into Special Camps'",
 'Texas: ‘Miraculous’ Baby Born 18 Weeks Early Home for the Holidays',
 '#keepkyle at ASU petition, both in support of Kyle, and in opposition to the students trying to have him barred.']

Negative headlines:

['Protesters Block Street Outside Hotel Where Conservative Activist Was Speaking',
 '"Unlike Trump I Don\'t Shock Allies": Biden On Expanding Omicron Travel Ban',
 'Michigan school shooting: Crumbley lawyer says gun was locked up, accuses prosecuto